<a href="https://colab.research.google.com/github/yogesh-bhattarai/100_days_of_deeplearning/blob/main/day_26(functional_api_).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [12]:
!kaggle datasets download -d jangedoo/utkface-new

 99% 329M/331M [00:16<00:00, 22.2MB/s]
100% 331M/331M [00:16<00:00, 21.4MB/s]


In [13]:
import zipfile
zip = zipfile.ZipFile("/content/utkface-new.zip",'r')
zip.extractall("/content")
zip.close()

In [14]:
import os
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator

In [15]:
folder_path='/content/utkface_aligned_cropped/UTKFace/'

In [16]:
age=[]
gender=[]
img_path=[]
for file in os.listdir(folder_path):
  age.append(int(file.split('_')[0]))
  gender.append(int(file.split('_')[1]))
  img_path.append(file)

In [17]:
age

[26,
 2,
 11,
 8,
 28,
 38,
 56,
 36,
 23,
 26,
 45,
 27,
 53,
 52,
 4,
 16,
 81,
 50,
 72,
 23,
 25,
 25,
 28,
 25,
 26,
 18,
 8,
 45,
 24,
 1,
 28,
 18,
 26,
 40,
 28,
 17,
 26,
 36,
 86,
 19,
 37,
 55,
 3,
 62,
 22,
 51,
 2,
 7,
 21,
 56,
 40,
 40,
 60,
 52,
 39,
 26,
 68,
 14,
 26,
 1,
 24,
 25,
 17,
 65,
 26,
 40,
 1,
 36,
 24,
 25,
 35,
 54,
 26,
 78,
 30,
 84,
 26,
 46,
 24,
 35,
 1,
 28,
 26,
 20,
 31,
 68,
 38,
 1,
 26,
 16,
 42,
 7,
 55,
 40,
 48,
 40,
 11,
 26,
 86,
 75,
 40,
 35,
 32,
 54,
 1,
 24,
 26,
 23,
 22,
 28,
 21,
 36,
 39,
 68,
 65,
 56,
 35,
 37,
 64,
 28,
 20,
 34,
 86,
 24,
 2,
 105,
 39,
 30,
 6,
 76,
 21,
 31,
 20,
 26,
 36,
 60,
 53,
 35,
 1,
 22,
 8,
 26,
 41,
 22,
 49,
 26,
 36,
 14,
 50,
 36,
 10,
 27,
 27,
 34,
 27,
 48,
 26,
 31,
 22,
 1,
 44,
 25,
 25,
 31,
 61,
 26,
 25,
 50,
 52,
 1,
 26,
 46,
 70,
 34,
 40,
 45,
 35,
 27,
 43,
 26,
 27,
 26,
 32,
 53,
 28,
 116,
 29,
 61,
 37,
 3,
 39,
 30,
 35,
 13,
 25,
 10,
 1,
 56,
 32,
 32,
 35,
 35,
 54,
 25,


In [18]:
len(age)

23708

In [19]:
df=pd.DataFrame({
    'age':age,
    'gender':gender,
    'img':img_path,
})

In [20]:
df.head()

,age,gender,img
0,26,1,26_1_3_20170104232413655.jpg.chip.jpg
1,2,0,2_0_4_20161221195155711.jpg.chip.jpg
2,11,1,11_1_2_20170104005111615.jpg.chip.jpg
3,8,1,8_1_0_20170109204933562.jpg.chip.jpg
4,28,0,28_0_4_20170109140345949.jpg.chip.jpg


In [21]:
train_df = df.sample(frac=1,random_state=0).iloc[:20000]
test_df = df.sample(frac=1,random_state=0).iloc[20000:]

In [22]:
train_df.shape

(20000, 3)

In [23]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [24]:
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    directory=folder_path,
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    target_size=(200,200),
                                                    class_mode='multi_output')

test_generator = test_datagen.flow_from_dataframe(test_df,
                                                    directory=folder_path,
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    target_size=(200,200),
                                                  class_mode='multi_output')

Found 20000 validated image filenames.
Found 3708 validated image filenames.


In [25]:
from keras.applications.resnet50 import ResNet50
from keras.layers import *
from keras.models import Model

In [26]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

94765736/94765736 [==============================] - 5s 0us/step


In [27]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

resnet.trainable=False

output = resnet.layers[-1].output

flatten = Flatten()(output)

dense1 = Dense(512, activation='relu')(flatten)
dense2 = Dense(512,activation='relu')(flatten)

dense3 = Dense(512,activation='relu')(dense1)
dense4 = Dense(512,activation='relu')(dense2)

output1 = Dense(1,activation='linear',name='age')(dense3)
output2 = Dense(1,activation='sigmoid',name='gender')(dense4)

In [28]:
model = Model(inputs=resnet.input,outputs=[output1,output2])

In [29]:
model.compile(optimizer='adam', loss={'age': 'mae', 'gender': 'binary_crossentropy'}, metrics={'age': 'mae', 'gender': 'accuracy'},loss_weights={'age':1,'gender':99})

In [ ]:
model.fit(train_generator, batch_size=32, epochs=10, validation_data=test_generator)

Epoch 1/10
625/625 [==============================] - 258s 391ms/step - loss: 99.9874 - age_loss: 15.4525 - gender_loss: 0.8539 - age_mae: 15.4525 - gender_accuracy: 0.5159 - val_loss: 83.1070 - val_age_loss: 14.5284 - val_gender_loss: 0.6927 - val_age_mae: 14.5284 - val_gender_accuracy: 0.5194
Epoch 2/10
241/625 [==========>...................] - ETA: 2:23 - loss: 84.0709 - age_loss: 15.2427 - gender_loss: 0.6952 - age_mae: 15.2427 - gender_accuracy: 0.5310